In [235]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import geopandas as gpd
import random
from scipy.stats import truncnorm, norm
from collections import defaultdict
import math
from shapely.ops import transform
from pyproj import Geod
import pyarrow
from datetime import datetime
import sys
import importlib
from dataclasses import dataclass
import yaml
from typing import Union

In [4]:
# добавляем папку modules в sys.path, чтобы импортировать функции созданные ранее

# sys.path.append(os.path.abspath(os.path.join("..", "modules")))

In [501]:
# Функции генерации паттернов времени и базовые функции времени созданные ранее
import data_generator.fraud.time
import data_generator.fraud.txndata
import data_generator.configs
importlib.reload(data_generator.fraud.time)
importlib.reload(data_generator.fraud.txndata)
importlib.reload(data_generator.configs)
from data_generator.general_time import *
from data_generator.utils import build_transaction
from data_generator.fraud.txndata import FraudTxnPartData, TransAmount, sample_category
from data_generator.fraud.time import derive_from_last_time
from data_generator.configs import PurchBehaviorHandler

In [10]:
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', None)

In [7]:
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01\\notebooks'

In [8]:
os.chdir("..")

In [9]:
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01'

## Загрузка конфигураций

In [1049]:
# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

# Доп. данные для генерации фрод транзакций

**Загрузка данных:**
- оффлайн мерчантов
- онлайн мерчантов
- девайсов клиентов и мошенников
- городов с полигонами
- городов с координатами центров
- клиентов
- фрод IP адресов
- категорий и их характеристик
- категорий для дропов покупателей
- Счетов клиентов
- Внешних счетов

In [612]:
offline_merchants = gpd.read_file("./data/cleaned_data/offline_merchants_points.gpkg")
online_merchant_ids = pd.read_csv("./data/cleaned_data/online_merchant_ids.csv").iloc[:, 0] # нужны в виде серии
client_devices = pd.read_csv("./data/cleaned_data/client_devices.csv")
fraud_devices = pd.read_csv("./data/cleaned_data/fraud_devices.csv")
districts_ru = gpd.read_file("./data/cleaned_data/district_ru.gpkg")
area_centers = gpd.read_file("./data/cleaned_data/area_centers.gpkg")
clients_with_geo = gpd.read_file("./data/cleaned_data/clients_with_geo.gpkg") 
fraud_ips = gpd.read_file("./data/cleaned_data/fraud_ips.gpkg")
cat_stats_full = pd.read_csv("./data/cleaned_data/cat_stats_full.csv")
cat_fraud_amts = pd.read_csv("./data/cleaned_data/cat_fraud_amts.csv")
drop_purch_cats = pd.read_csv("./data/cleaned_data/drop_purch_cats.csv")
accounts = pd.read_csv("./data/generated_data/accounts.csv")
outer_accounts = pd.read_csv("./data/generated_data/outer_accounts.csv").iloc[:, 0] # нужны в виде серии

**Датафрейм под транзакции. Этап тестов**<br>
Создадим пустой датафрейм под транзакции. Будем копировать и использовать его при тестах функций.

In [13]:
transactions = pd.DataFrame(
            {"client_id": pd.Series(dtype="int64"),
            "txn_time": pd.Series(dtype="datetime64[ns]"),
             "unix_time":pd.Series(dtype="int64"),
            "amount": pd.Series(dtype="float64"),
            "type": pd.Series(dtype="string"),
            "channel": pd.Series(dtype="string"),
            "category": pd.Series(dtype="string"),
            "online":pd.Series(dtype="bool"),
            "merchant_id":pd.Series(dtype="int64"),
             "trans_city":pd.Series(dtype="string"),
            "trans_lat":pd.Series(dtype="float64"),
             "trans_lon":pd.Series(dtype="float64"),
            "trans_ip":pd.Series(dtype="string"),
             "device_id":pd.Series(dtype="int64"),
             "account": pd.Series(dtype="int64"),
            "is_fraud": pd.Series(dtype="bool"),
             "is_suspicious": pd.Series(dtype="bool"),
            "status":pd.Series(dtype="string"),
            "rule":pd.Series(dtype="string")})
transactions

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule


**Функция получения случайных значений из `truncnorm` распределения**
- пойдет в модуль `utils`

In [14]:
def get_values_from_truncnorm(low_bound, high_bound, mean, std, size=1):
    """
    Сгенерировать массив чисел из обрезанного нормального распределения.
    Можно сгенерировать массив с одним числом
    ------------
    low_bound - float, int. Нижняя граница значений
    high_bound - float, int. Верхняя граница значений 
    mean - float, int. Среднее
    std - float, int. Стандартное отклонение
    size - Количество чисел в возвращаемом массиве
    ------------
    Возвращает np.ndarray
    """
    return truncnorm.rvs((low_bound - mean) / std, (high_bound - mean) / std, loc=mean, scale=std, size=size)

## Класс `DropDistributorCfg` конфиги для дропов распределителей
- конфигурации на основании которых будут генерироваться транзакции дропов распределителей,
  которые заняты распределением полученных денег: переводами, снятиями, покупкой криптовалюты.

In [52]:
# ----------- ВСТАВИТЬ ГОТОВЫЙ КЛАСС СЮДА -------------------------

## Класс `DropPurchaserCfg` конфиги для дропов покупателей
- конфигурации на основании которых будут генерироваться транзакции дропов покупателей,
  которые заняты покупкой товаров на полученные деньги.

In [ ]:
# ----------- ВСТАВИТЬ ГОТОВЫЙ КЛАСС СЮДА -------------------------

## Класс `DropConfigBuilder`
- создает объект `DropDistributorCfg` или `DropPurchaserCfg`

In [ ]:
# ----------- ВСТАВИТЬ ГОТОВЫЙ КЛАСС СЮДА -------------------------

In [1101]:
# Временно

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [1154]:
import data_generator.indev
import data_generator.configs
importlib.reload(data_generator.indev)
importlib.reload(data_generator.configs)
from data_generator.indev import DropConfigBuilder
from data_generator.configs import DropDistributorCfg

### **Тест `DropConfigBuilder`**

In [1145]:
drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)

In [1107]:
clients_sample = gpd.read_file("./data/cleaned_data/clients_sample.gpkg")

In [1108]:
# Клиенты незадействованные ранее в легальных транзакциях и compromised фроде

not_used_clients = clients_with_geo.loc[~clients_with_geo.client_id.isin(clients_sample.client_id)].copy()

**Тест `get_clients_for_drops`**

**генерируем `purchaser`-ов**  
Файл с `distributor`-ами существует

In [1125]:
dist_count = drop_cfg_buid.estimate_drops_count(drop_type="distributor")

In [1114]:
dist_drops = not_used_clients.sample(n=dist_count).reset_index(drop=True)

In [1115]:
dist_drops.to_file("./data/generated_data/dist_drops.gpkg", layer='layer_name', driver="GPKG")

In [1117]:
drop_cfg_build.get_clients_for_drops(drop_type="purchaser")
print(drop_cfg_build.drops.shape[0])
drop_cfg_build.drops.head()

28


,client_id,district_id,birth_date,sex,region,area,timezone,lat,lon,population,home_ip,geometry
0,1829,51,1942-08-16,female,Дагестан,Махачкала,UTC+3,42.984857,47.504630,577990,2.60.6.189,"MULTIPOLYGON (((47.51911 42.90425, 47.53096 42..."
1,4762,76,1971-02-13,female,Тюменская,Тюмень,UTC+5,57.153082,65.534312,581758,2.60.17.151,"MULTIPOLYGON (((65.26579 57.16358, 65.26877 57..."
2,3812,1,1967-02-08,male,Москва,Москва,UTC+3,55.753879,37.620373,11514330,2.60.14.25,"MULTIPOLYGON (((37.2905 55.80199, 37.29542 55...."
3,622,74,1931-11-23,female,Санкт-Петербург,Санкт-Петербург,UTC+3,59.939125,30.315822,4848742,2.60.2.81,"MULTIPOLYGON (((30.04334 59.76418, 30.04535 59..."
4,1496,1,1942-06-08,female,Москва,Москва,UTC+3,55.753879,37.620373,11514330,2.60.5.137,"MULTIPOLYGON (((37.2905 55.80199, 37.29542 55...."


In [1118]:
purch_drops = drop_cfg_build.drops

In [1120]:
samp_and_dist = pd.concat([clients_sample, dist_drops], ignore_index=True)
assert samp_and_dist.loc[samp_and_dist.client_id.isin(purch_drops.client_id)].empty

**генерируем `purchaser`-ов**  
Файл с `distributor`-ами НЕ существует

In [1127]:
path1 = "./data/generated_data/dist_drops.gpkg"
if os.path.exists(path1):
    os.remove(path1)
drop_cfg_build.get_clients_for_drops(drop_type="purchaser")
print(drop_cfg_build.drops.shape[0])
drop_cfg_build.drops.head()

28


,client_id,district_id,birth_date,sex,region,area,timezone,lat,lon,population,home_ip,geometry
0,1179,1,1930-10-05,female,Москва,Москва,UTC+3,55.753879,37.620373,11514330,2.60.4.100,"MULTIPOLYGON (((37.2905 55.80199, 37.29542 55...."
1,1407,29,1941-08-04,male,Архангельская,Архангельск,UTC+3,64.539299,40.517008,348716,2.60.5.53,"MULTIPOLYGON (((40.28494 64.53094, 40.28874 64..."
2,11662,76,1972-04-03,male,Тюменская,Тюмень,UTC+5,57.153082,65.534312,581758,2.60.20.72,"MULTIPOLYGON (((65.26579 57.16358, 65.26877 57..."
3,5,5,1960-07-03,female,Ростовская,Ростов-на-Дону,UTC+3,47.222436,39.718787,1091544,2.60.0.5,"MULTIPOLYGON (((39.47099 47.20487, 39.48381 47..."
4,3883,18,1965-09-27,male,Рязанская,Рязань,UTC+3,54.625457,39.735999,525062,2.60.14.91,"MULTIPOLYGON (((39.53667 54.6385, 39.53677 54...."


In [1128]:
purch_drops2 = drop_cfg_build.drops

In [1129]:
assert clients_sample.loc[clients_sample.client_id.isin(purch_drops2.client_id)].empty

**генерируем `distributor-ов`**  
Файл с `purchaser`-ами существует

In [1135]:
purch_count = drop_cfg_buid.estimate_drops_count(drop_type="purchaser")
purch_drops = not_used_clients.sample(n=purch_count).reset_index(drop=True)
dist_drops.to_file("./data/generated_data/dist_drops.gpkg", layer='layer_name', driver="GPKG")

In [1136]:
path2 = "./data/generated_data/purchase_drops.gpkg"
assert os.path.exists(path2), "File does not exist"

drop_cfg_build.get_clients_for_drops(drop_type="distributor")
print(drop_cfg_build.drops.shape[0])
drop_cfg_build.drops.head()

28


,client_id,district_id,birth_date,sex,region,area,timezone,lat,lon,population,home_ip,geometry
0,12739,23,1958-01-17,male,Владимирская,Владимир,UTC+3,56.128156,40.408299,348256,2.60.20.159,"MULTIPOLYGON (((40.16496 56.12132, 40.16772 56..."
1,2404,27,1977-07-05,female,Брянская,Брянск,UTC+3,53.242007,34.365272,415640,2.60.8.230,"MULTIPOLYGON (((34.1933 53.32201, 34.19526 53...."
2,525,30,1969-08-14,male,Вологодская,Вологда,UTC+3,59.248419,39.835646,301642,2.60.1.246,"MULTIPOLYGON (((39.73325 59.2304, 39.73337 59...."
3,3880,29,1961-08-24,female,Архангельская,Архангельск,UTC+3,64.539299,40.517008,348716,2.60.14.88,"MULTIPOLYGON (((40.28494 64.53094, 40.28874 64..."
4,4072,45,1941-05-08,male,Курганская,Курган,UTC+5,55.444345,65.316134,333640,2.60.15.15,"MULTIPOLYGON (((65.11762 55.44839, 65.13733 55..."


In [1137]:
dist_drops1 = drop_cfg_build.drops

In [1139]:
samp_and_purch = pd.concat([clients_sample, purch_drops], ignore_index=True)
assert samp_and_purch.loc[samp_and_purch.client_id.isin(dist_drops1.client_id)].empty

**генерируем `distributor-ов`**  
Файл с `purchaser`-ами НЕ существует

In [1140]:
path2 = "./data/generated_data/purchase_drops.gpkg"
if os.path.exists(path2):
    os.remove(path2)
drop_cfg_build.get_clients_for_drops(drop_type="distributor")
print(drop_cfg_build.drops.shape[0])
drop_cfg_build.drops.head()

28


,client_id,district_id,birth_date,sex,region,area,timezone,lat,lon,population,home_ip,geometry
0,4198,23,1954-10-17,female,Владимирская,Владимир,UTC+3,56.128156,40.408299,348256,2.60.15.132,"MULTIPOLYGON (((40.16496 56.12132, 40.16772 56..."
1,2022,56,1939-11-12,male,Крым,Симферополь,UTC+3,44.948265,34.100117,362366,2.60.7.117,"MULTIPOLYGON (((34.02892 44.93592, 34.03005 44..."
2,3654,25,1924-03-01,female,Ханты-Мансийский Автономный округ - Югра,Сургут,UTC+5,61.253977,73.396173,306703,2.60.13.135,"MULTIPOLYGON (((73.23552 61.28064, 73.2369 61...."
3,3421,20,1947-03-05,male,Иркутская,Братск,UTC+8,56.151395,101.633989,246348,2.60.12.166,"MULTIPOLYGON (((101.35303 56.14807, 101.35323 ..."
4,2887,2,1961-01-29,female,Мордовия,Саранск,UTC+3,54.180760,45.186226,297425,2.60.10.172,"MULTIPOLYGON (((45.05548 54.18571, 45.06143 54..."


In [1141]:
dist_drops2 = drop_cfg_build.drops

In [1142]:
assert clients_sample.loc[clients_sample.client_id.isin(dist_drops2.client_id)].empty

**Тест `build_dist_cfg`**  
**создание объекта `DropDistributorCfg`**

In [1159]:
dist_configs = drop_cfg_build.build_dist_cfg()

Конфиги загружаются временно тут

In [1067]:
# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

## Объект датакласса `DropDistributorCfg` конфиги для дропов распределителей

In [1015]:
# Импорт на время разработки
import data_generator.configs
importlib.reload(data_generator.configs)
from data_generator.configs import DropDistributorCfg

**Доп. конфиги из `drop.yaml`**

In [1040]:
# временно

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [1041]:
# Параметры обозначенные в config/drops.yaml

dist_cfg = drops_cfg["distributor"]
dist_in_lim = dist_cfg["in_lim"]
dist_out_im = dist_cfg["out_lim"]
period_in_lim = dist_cfg["period_in_lim"]
period_out_lim = dist_cfg["period_out_lim"]
dist_lag_int = drops_cfg["time"]["lag_interval"]
two_way_delta = drops_cfg["time"]["two_way_delta"]
pos_delta = drops_cfg["time"]["pos_delta"]
split_rate = dist_cfg["split_rate"]
chunks = dist_cfg["chunks"]
dist_inbound = drops_cfg["inbound_amt"]
dist_round = dist_cfg["round"]
dist_trf_max = dist_cfg["trf_max"]
dist_reduce = dist_cfg["reduce_share"]
dist_att = dist_cfg["attempts"]
to_drops = dist_cfg["to_drops"]
crypto_rate = dist_cfg["crypto_rate"]

In [1042]:
dist_configs = DropDistributorCfg(clients=dist_drop_clients, timestamps=timestamps, accounts=accounts, \
                                  outer_accounts=outer_accounts, client_devices=client_devices, \
                                  online_merchant_ids=online_merchant_ids, cities=districts_ru, in_lim=dist_in_lim, 
                                  out_lim=dist_out_im, period_in_lim=period_in_lim, period_out_lim=period_out_lim, \
                                  lag_interval=dist_lag_int, two_way_delta=two_way_delta, pos_delta=pos_delta, \
                                  split_rate=split_rate, chunks=chunks, inbound_amt=dist_inbound, round=dist_round, \
                                  trf_max=dist_trf_max, reduce_share=dist_reduce, attempts=dist_att, to_drops=to_drops, \
                                  crypto_rate=crypto_rate
                                 )

## Класс `DropAccountHandler`

In [484]:
class DropAccountHandler:
    """
    Генератор номеров счетов входящих/исходящих транзакций.
    Учет использованных счетов.
    """
    def __init__(self, configs: DropConfigs, client_id, used_accounts=pd.Series(name="account_id")):
        """
        configs - pd.DataFrame. Данные для создания транзакций: номера счетов клиентов и внешних счетов, и др.
        client_id - int. id текущего дропа.
        used_accounts - pd.Series. Счета на которые дропы уже отправляли деньги.
        """
        self.accounts = configs.accounts
        self.outer_accounts = configs.outer_accounts
        self.client_id = client_id
        # self.account = 0
        self.account = configs.accounts.loc[configs.accounts.client_id == client_id, "account_id"].iat[0]
        self.used_accounts = used_accounts

    def get_account(self, to_drop):
        """
        Номер счета входящего/исходящего перевода
        to_drop - bool. Перевод другому дропу в нашем банке или нет.
        """
        # Если отправляем/получаем из другого банка.  
        if not to_drop:
            # Семплируем номер внешнего счета который еще не использовался
            account = self.outer_accounts.loc[~(self.outer_accounts.isin(self.used_accounts))].sample(1).iat[0]
            # Добавляем этот счет в использованные как последнюю запись в серии
            self.used_accounts.loc[self.used_accounts.shape[0]] = account
            return account
        
        # Если надо отправить другому дропу в нашем банке. При условии что есть другие дропы на текущий момент
        # Фильтруем accounts исключая свой счет и использованные счета, и выбирая дропов. Для случая если to_drop
        drop_accounts = self.accounts.loc[(self.accounts.client_id != self.client_id) & (self.accounts.is_drop == True) \
                                          & ~(self.accounts.account_id.isin(self.used_accounts))]
        
        # Если счетов дропов ещё нет. Берем внешний неиспользованный счет
        if drop_accounts.empty:
            account = self.outer_accounts.loc[~(self.outer_accounts.isin(self.used_accounts))].sample(1).iat[0]
            self.used_accounts.loc[self.used_accounts.shape[0]] = account
            return account

        # Дропы есть
        account = drop_accounts.account_id.sample(1).iat[0]
        # Добавляем этот счет в использованные как последнюю запись в серии
        self.used_accounts.loc[self.used_accounts.shape[0]] = account
        return account


    def reset_cache(self):
        """
        Сброс кэшированных значений.
        Это только использованные номера счетов
        """
        self.used_accounts = pd.Series(name="account_id")

**Тест `DropAccountHandler`**

In [1266]:
# импорт на время разработки

import data_generator.fraud.drops.base
import data_generator.indev
import data_generator.configs

importlib.reload(data_generator.fraud.drops.base)
importlib.reload(data_generator.indev)
importlib.reload(data_generator.configs)

from data_generator.fraud.drops.base import DropAccountHandler
from data_generator.indev import DropConfigBuilder
from data_generator.configs import DropDistributorCfg

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
dist_configs = drop_cfg_build.build_dist_cfg()

In [1279]:
drop_acc_hand1 = DropAccountHandler(configs=dist_configs)

In [1281]:
drop_acc_hand1.client_id = 1

In [1282]:
drop_acc_hand1.get_account(own=True)
drop_acc_hand1.account

np.int64(10000)

In [1270]:
drop_acc_hand1.get_account()

np.int64(21522)

In [1271]:
drop_acc_hand1.used_accounts

0    21522
Name: account_id, dtype: int64

In [1272]:
drop_acc_hand1.label_drop()

In [1273]:
drop_acc_hand1.accounts.head(2)

,client_id,account_id,is_drop
0,1,10000,True
1,2,10001,False


In [1274]:
drop_acc_hand1.client_id = 99

In [1275]:
drop_acc_hand1.get_account(own=True)
drop_acc_hand1.label_drop()

In [1276]:
drop_acc_hand1.accounts.query("client_id == 99")

,client_id,account_id,is_drop
93,99,10093,True


**`to_drop=True`**  нет дропов в self.accounts  (кроме текущего дропа)

In [1277]:
own_id = drop_acc_hand1.client_id

In [1283]:
drop_acc_hand1.accounts.query("client_id != @own_id and is_drop == True")

,client_id,account_id,is_drop


In [1181]:
drop_acc_hand1.reset_cache()
outer_acc1 = drop_acc_hand1.get_account(to_drop=True)
drop_acc_hand1.outer_accounts.loc[drop_acc_hand1.outer_accounts == outer_acc1]

3410    18779
Name: account_id, dtype: int64

In [1182]:
drop_acc_hand1.used_accounts

0    18779
Name: account_id, dtype: int64

**`to_drop=True`** кол-во дропов в self.accounts меньше чем self.min_drops

In [710]:
drop_acc_hand1.reset_cache()
min_drops = dist_configs.to_drops["min_drops"]
print(min_drops)
drop_acc_hand1.accounts["is_drop"] = False
drop_acc_hand1.get_account(own=True)

accs_samp = drop_acc_hand1.accounts.query("client_id != @own_id").client_id.sample(n=min_drops - 1)
drop_acc_hand1.accounts.loc[drop_acc_hand1.accounts.client_id.isin(accs_samp), "is_drop"] = True
drop_acc_hand1.accounts.query("client_id != @own_id and is_drop == True").shape[0]

6


5

In [711]:
outer_acc2 = drop_acc_hand1.get_account(to_drop=True)
outer_acc2

np.int64(21917)

In [712]:
drop_acc_hand1.outer_accounts.loc[drop_acc_hand1.outer_accounts == outer_acc2]

6548    21917
Name: account_id, dtype: int64

In [713]:
drop_acc_hand1.used_accounts

0    21917
Name: account_id, dtype: int64

**`to_drop=True`** кол-во дропов в self.accounts `>=` self.min_drops

In [714]:
drop_acc_hand1.reset_cache()
min_drops = dist_configs.to_drops["min_drops"]
print(min_drops)
drop_acc_hand1.accounts["is_drop"] = False
drop_acc_hand1.get_account(own=True)

accs_samp3 = drop_acc_hand1.accounts.query("client_id != @own_id").client_id.sample(n=min_drops)
drop_acc_hand1.accounts.loc[drop_acc_hand1.accounts.client_id.isin(accs_samp3), "is_drop"] = True
drop_acc_hand1.accounts.query("client_id != @own_id and is_drop == True").shape[0]

6


6

In [715]:
drop_acc1 = drop_acc_hand1.get_account(to_drop=True)
drop_acc1

np.int64(13694)

In [717]:
drop_acc_hand1.accounts.loc[drop_acc_hand1.accounts.account_id == drop_acc1]

,client_id,account_id,is_drop
3694,3904,13694,True


## Класс `DropAmountHandler`

In [ ]:
# class DropAmountHandler: 
#     """
#     Генератор сумм входящих/исходящих транзакций, сумм снятий.
#     Управление балансом текущего дропа.
#     -------------
#     Атрибуты:
#     balance: float, int. Текущий баланс дропа. По умолчанию 0.
#     batch_txns: int. Счетчик транзакций сделанных в рамках распределения полученной партии денег.
#                       По умолчанию 0.
#     chunk_size: int, float. Последний созданный размер части баланса для перевода по частям
#                              По умолчанию 0.
#     chunks: dict. Содержит ключи:
#         atm_min: int. Минимальная сумма для снятий в банкомате.
#         atm_share: float. Доля от баланса, которую дроп снимает в случае снятия в банкомате
#         low: int. Минимальная сумма исходящего перевода частями.
#         high: int. Максимальная сумма исходящего перевода частями.
#         step: int. Шаг возможных сумм. Чем меньше шаг, тем больше вариантов.
#         rand_rate: float. От 0 до 1.
#                    Процент случаев, когда каждый НЕ первый чанк будет случайным и не зависеть от предыдущего.
#                    Но возможны случайные совпадаения с предыдущим размером чанка.
#                    Доля случайных размеров подряд будет:
#                    p(c) - вероятность взять определенный размер (зависит от размера выборки чанков)
#                    p(r) - rand_rate
#                    p(r) - (p(r) * p(c)). Например p(r) = 0.9; и 5 вариантов размеров чанка - p(c) = 0.20
#                    0.9 - (0.9 * 0.2) = 0.72
#                    В около 72% случаев размеры чанков не будут подряд одинаковыми 
#     inbound_amt: dict. Настройки для сумм входящих транзакций. Содержит ключи:
#         low: int
#         high: int
#         mean: int
#         std: int
#     round: int. Округление целой части сумм транзакций. Напр. 500 значит что суммы будут кратны 500 - кончаться на 500 или 000                  
#     """

#     def __init__(self, configs: DropDistributorCfg | DropPurchaserCfg):
#         """
#         configs: DropDistributorCfg | DropPurchaserCfg. Данные на основании, которых генерируются транзакции.
#                  Отсюда берутся: atm_min, atm_share, min, step, rand_rate.
#         """
#         self.balance = 0
#         self.batch_txns = 0
#         self.chunk_size = 0
#         self.chunks = configs.chunks.copy()
#         self.inbound_amt = configs.inbound_amt.copy()
#         self.round = configs.round
#         # self.atm_min = configs.chunks["atm_min"]
#         # self.atm_share = configs.chunks["atm_share"]
#         # self.min = configs.chunks["min"]
#         # self.max = configs.chunks["max"]
#         # self.step = configs.chunks["step"]
#         # self.rand_rate = configs.chunks["rand_rate"]

#     def update_balance(self, amount, receive=False, declined=False):
#         """
#         Увеличить/уменьшить баланс на указанную сумму
#         -------------------
#         amount - float, int.
#         receive - bool. Входящая ли транзакция. Прибавлять сумму или отнимать.
#         declined - bool. Отклонена ли транзакция или одобрена.
#         """
#         # Не обновлять баланс если транзакция отклонена.
#         if declined:
#             return
            
#         # Увеличить баланс   
#         if receive:
#             self.balance += amount
#             return
            
#         # Уменьшить баланс    
#         self.balance -= amount

#     def receive(self, declined):
#         """
#         Генерация суммы входящего перевода
#         --------------------------
#         declined - bool. Отклонена ли транзакция или одобрена
#         """
#         low = self.inbound_amt["low"]
#         high = self.inbound_amt["high"]
#         mean = self.inbound_amt["mean"]
#         std = self.inbound_amt["std"]

#         # Генерация суммы. Округление целой части при необходимости
#         amount = get_values_from_truncnorm(low_bound=low, high_bound=high, mean=mean, std=std)[0] // self.round * self.round
        
#         # Обновляем баланс если транзакция не отклонена
#         self.update_balance(amount=amount, receive=True, declined=declined)
        
#         return amount

#     def get_chunk_size(self, online=False):
#         """
#         Вернуть случайный размер суммы перевода для перевода по частям
#         либо вернуть долю от баланса для снятия/перевода по частям.
#         -------------------------------
#         online - bool. Онлайн или оффлайн. Перевод или банкомат. Если банкомат, то снимается доля self.chunks["atm_share"] от баланса, 
#                  но не меньше self.chunks["atm_min"]
#         --------------------
#         Возвращает np.int64
#         Результат кэшируется в self.chunk_size
#         """
#         # Если это не первая транзакция в серии транзакции для одной полученной дропом суммы
#         # И случайное число больше rand_rate, то просто возвращаем ранее созданный размер чанка
#         rand_rate = self.chunks["rand_rate"]
#         if self.batch_txns != 0 and np.random.uniform(0,1) > rand_rate:
#             return self.chunk_size

#         # Если снятие
#         if not online:
#             atm_min = self.chunks["atm_min"]
#             atm_share = self.chunks["atm_share"]
#             self.chunk_size = max(atm_min, self.balance * atm_share // self.round * self.round)
#             return self.chunk_size
        
#         # Если перевод. 
#         # Берем лимиты под генерацию массива чанков, в зависимости от
#         # полученной дропом суммы
#         small = self.chunks["rcvd_small"]
#         medium = self.chunks["rcvd_medium"] 
#         large = self.chunks["rcvd_large"]
#         step = self.chunks["step"]

#         if self.balance <= small["limit"]:
#             low = min(self.balance, small["min"]) # но не больше суммы на балансе
#             high = min(self.balance, small["max"]) # но не больше суммы на балансе

#         elif self.balance <= medium["limit"]:
#             low = min(self.balance, medium["min"])
#             high = min(self.balance, medium["max"])

#         else:
#             low = min(self.balance, large["min"])
#             high = min(self.balance, large["max"])
   
#         # прибавим шаг к максимуму, чтобы было понятнее передавать аргументы в конфиге 
#         # и не учитывать исключение значения stop в np.arange
#         sampling_array = np.arange(low, high + step, step)
#         # Если чанк больше бал
#         self.chunk_size = np.random.choice(sampling_array)
#         return self.chunk_size

**Тест `DropAmountHandler`**

In [1183]:
# импорт на время разработки

import data_generator.fraud.drops.base
import data_generator.indev
import data_generator.configs

importlib.reload(data_generator.indev)
importlib.reload(data_generator.configs)
importlib.reload(data_generator.fraud.drops.base)

from data_generator.indev import DropConfigBuilder
from data_generator.configs import DropDistributorCfg
from data_generator.fraud.drops.base import DropAmountHandler

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
dist_configs = drop_cfg_build.build_dist_cfg()
drop_amt1 = DropAmountHandler(configs=dist_configs)

In [1184]:
drop_amt1.chunks

{'atm_min': 10000,
 'atm_share': {'min': 0.3, 'max': 0.7},
 'step': 1000,
 'rand_rate': 0.9,
 'rcvd_small': {'limit': 10000, 'min': 3000, 'max': 10000},
 'rcvd_medium': {'limit': 30000, 'min': 5000, 'max': 25000},
 'rcvd_large': {'min': 10000, 'max': 40000}}

In [1185]:
drop_amt1.balance, drop_amt1.batch_txns, drop_amt1.chunk_size

(0, 0, 0)

In [1186]:
drop_amt1.get_atm_share

0.5205433170302771

In [1187]:
# drop_amt1.chunks, drop_amt1.receive, drop_amt1.round

In [1188]:
drop_amt1.reset_cache()
drop_amt1.balance

0

In [1189]:
drop_amt1.inbound_amt

{'low': 5000, 'high': 100000, 'mean': 30000, 'std': 20000}

In [1190]:
drop_amt1.balance = 50000
drop_amt1.get_chunk_size(online=False)

24800.0

In [1191]:
drop_amt1.reset_cache()
received_list = [drop_amt1.receive(declined=True) for _ in range(10000)]
rcvd_amts_df = pd.DataFrame(pd.Series(received_list, name="rcvd_amt"))
print(f"balance: {drop_amt1.balance}")
rcvd_amts_df.rcvd_amt.agg(["min","max", "mean"])

balance: 50000


min      5000.00
max     98200.00
mean    33631.22
Name: rcvd_amt, dtype: float64

In [1192]:
drop_amt1.receive(declined=False)

np.float64(14000.0)

In [1193]:
drop_amt1.balance

np.float64(64000.0)

In [1194]:
drop_amt1.receive(declined=False)

np.float64(25400.0)

In [1195]:
drop_amt1.balance

np.float64(89400.0)

In [1196]:
print("before: ", drop_amt1.balance)
drop_amt1.update_balance(amount=7000, receive=False, declined=True)
drop_amt1.balance

before:  89400.0


np.float64(89400.0)

**Тест `chunk_size`**

In [689]:
# Значения атрибутов отличные от прописанных в конфиге - закомментить/раскомментить при надобности
# drop_amt1.chunks["low"] = 10000
# drop_amt1.chunks["high"] = 45000
# drop_amt1.chunks["step"] = 1000
# drop_amt1.chunks["atm_min"] = 15000
# drop_amt1.chunks["atm_share"] = 0.4
# drop_amt1.chunks["rand_rate"] = 0.5

In [1197]:
drop_amt1.balance = 10000
drop_amt1.batch_txns = 1
drop_amt1.get_chunk_size(online=False), drop_amt1.chunk_size

(10000, 10000)

In [1198]:
drop_amt1.batch_txns = 1
# drop_amt1.chunk_size = 20000
drop_amt1.balance = drop_amt1.chunks["rcvd_small"]["limit"]
chunks_list = [drop_amt1.get_chunk_size(online=True) for _ in range(1000)]
chunks_df = pd.DataFrame(pd.Series(chunks_list, name="chunk_size"))
chunks_df.head()

,chunk_size
0,10000
1,6000
2,3000
3,3000
4,9000


In [762]:
chunks_df["prev_size"] = chunks_df.chunk_size.shift(1)
chunks_df["eq_to_prev"] = chunks_df["chunk_size"] == chunks_df["prev_size"]
chunks_df.head(3)

,chunk_size,prev_size,eq_to_prev
0,7000,NaN,False
1,4000,7000.0,False
2,4000,4000.0,True


In [105]:
rand_rate1 = drop_amt1.chunks["rand_rate"]

# Берем лимиты под генерацию массива чанков, в зависимости от
# полученной дропом суммы
small = drop_amt1.chunks["rcvd_small"]
medium = drop_amt1.chunks["rcvd_medium"] 
large = drop_amt1.chunks["rcvd_large"]

if drop_amt1.balance <= small["limit"]:
    low = small["min"]
    high = min(drop_amt1.balance, small["max"])

elif drop_amt1.balance <= medium["limit"]:
    low = medium["min"]
    high = min(drop_amt1.balance, medium["max"])

else:
    low = large["min"]
    high = min(drop_amt1.balance, large["max"])
    
step = drop_amt1.chunks["step"]
chunks_num = np.arange(start=low, stop=high + step, step=step).shape[0]
one_chunk_p = 1 / chunks_num
print(f"""balance: {drop_amt1.balance}
rand_rate: {rand_rate1}
chunks number: {chunks_num}
prob of taking one chunk: {one_chunk_p:.3f}
{rand_rate1 - (rand_rate1 * one_chunk_p):.3f} chunks won't be the same as a previous chunk size""")

balance: 58500.0
rand_rate: 0.9
chunks number: 31
prob of taking one chunk: 0.032
0.871 chunks won't be the same as a previous chunk size


In [764]:
chunks_df.eq_to_prev.value_counts(normalize=True)

eq_to_prev
False    0.788
True     0.212
Name: proportion, dtype: float64

In [757]:
chunks_df.chunk_size.agg(["min","max"])

min     5000
max    20000
Name: chunk_size, dtype: int64

**Тест `reduce_amt`**

In [1202]:
drop_amt1.balance = 10000
drop_amt1.first_decl = 10000
drop_amt1.last_amt = 7500
print("manual: ", drop_amt1.first_decl * drop_amt1.reduce_share)
drop_amt1.reduce_amt(online=True)

manual:  2500.0


5000.0

**тест `one_operation`**

In [ ]:
# chunks: # Настройки создания суммы, для распределения полученных денег по частям
# atm_min: 10000 # Минимальная сумма для снятий в банкомате.
# atm_share: 0.5 # Доля от баланса, которую дроп снимает в случае снятия в банкомате
# step: 1000 # Шаг возможных сумм. Чем меньше шаг, тем больше вариантов.
# rand_rate: 0.9 # В скольких случаях размер суммы должен быть семплирован, а не взят размер предыдущей транзакции
# rcvd_small: # Категория полученной дропом суммы
#   limit: 10000 # Категория все что "до" включительно
#   min: 3000 # # Минимальная сумма чанка для этой категории
#   max: 10000 # Максимальная сумма чанка для этой категории
# rcvd_medium:
#   limit: 30000
#   min: 5000
#   max: 25000
# rcvd_large: # порог не прописываем, т.к. это все что выше medium
#   min: 10000
#   max: 40000

In [1203]:
small = drop_amt1.chunks["rcvd_small"]
medium = drop_amt1.chunks["rcvd_medium"] 
large = drop_amt1.chunks["rcvd_large"]

**Ниже ячейка для кейсов:**
- Не отклонена. Онлайн. Частями. Small
- Не отклонена. Онлайн. Частями. Medium
- Не отклонена. Онлайн. Частями. Large

- `min_lim` всегда стоит `min_lim = small["min"]` т.к. баланс уменьшается и могут срабатывать условия для `small`, `medium`
- `drop_amt1.balance` для large ставить как `medium["limit"] + число` т.к. у large нет своего лимита, используется все что выше `medium["limit"]`

In [1204]:
i = 0
min_lim = small["min"]
max_lim = small["max"]

while i < 1000:
    drop_amt1.reset_cache()
    drop_amt1.balance = small["limit"]
    all_ops = []
    
    while drop_amt1.balance > 0:
        one_op1 = drop_amt1.one_operation(online=True, declined=False, in_chunks=True)
        all_ops.append(one_op1)
        i += 1
    all_ops_ser = pd.Series(all_ops)
    # для минимального берем серию без последнего элемента. Т.к. там обычно уже остаток, а не чанк
    if all_ops_ser.shape[0] == 1:
        assert all_ops_ser.min() >= min_lim, \
                            f"Min is lower than min limit. {all_ops_ser.min()}"
    else:
        assert all_ops_ser.iloc[0:(all_ops_ser.shape[0] - 1)].min() >= min_lim, \
                            f"Min is lower than min limit. {all_ops_ser.iloc[0:(all_ops_ser.shape[0] - 1)].min()}"
        
    assert all_ops_ser.max() <= max_lim, "Max is higher than min limit."

In [1205]:
all_ops_ser

0    5000
1    3000
2    2000
dtype: int64

In [1206]:
atm_min = drop_amt1.chunks["atm_min"]

In [1208]:
drop_amt1.reset_cache()
drop_amt1.balance = atm_min
print(drop_amt1.balance)
all_ops2 = []

while drop_amt1.balance > 0:
    one_op2 = drop_amt1.one_operation(online=False, declined=False, in_chunks=True)
    all_ops2.append(one_op2)
all_ops_ser = pd.Series(all_ops2)
all_ops_ser

10000


0    10000
dtype: int64

**Тест отклоненных транзакций. Онлайн/оффлайн**

In [1214]:
drop_amt1.chunks["atm_min"]

10000

In [1215]:
drop_amt1.reset_cache(life_end=True)
drop_amt1.balance = atm_min + 37000
print(drop_amt1.balance)
one_op_list1 = []
while True:
    # print(drop_amt1.first_decl)
    one_op1 = drop_amt1.one_operation(online=False, declined=True, in_chunks=True)
    
    one_op_list1.append(one_op1)
    if len(one_op_list1) == 10:
        break
print(drop_amt1.balance, "\n", one_op_list1)

47000
47000 
 [30300.0, 22800.0, 15300.0, 10000, 10000, 10000, 10000, 10000, 10000, 10000]


In [1216]:
cum_sub_df = pd.DataFrame()
cum_sub_df["amt"] = np.arange(0, 11, step=1)
cum_sub_df["amt"] = - (30300 * 0.25 // 100 * 100)
cum_sub_df.loc[0, "amt"] = 30300
cum_sub_df.amt.cumsum()

0     30300.0
1     22800.0
2     15300.0
3      7800.0
4       300.0
5     -7200.0
6    -14700.0
7    -22200.0
8    -29700.0
9    -37200.0
10   -44700.0
Name: amt, dtype: float64

## Класс `DropTimeHandler`
- управление временем транзакций дропа

In [238]:
# <------------------------- ГОТОВЫЙ КЛАСС ВСТАВИТЬ--------------------------

**Тест `DropTimeHandler`**

In [1217]:
# Временный reload для разработки
import data_generator.fraud.drops.base
import data_generator.indev
import data_generator.configs

importlib.reload(data_generator.indev)
importlib.reload(data_generator.configs)
importlib.reload(data_generator.fraud.drops.base)

from data_generator.indev import DropConfigBuilder
from data_generator.configs import DropDistributorCfg
from data_generator.fraud.drops.time import DropTimeHandler

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
dist_configs = drop_cfg_build.build_dist_cfg()
drop_time_hanler1 = DropTimeHandler(configs=dist_configs)

In [1218]:
two_way_deltas1 = pd.Series([drop_time_hanler1.get_time_delta(two_way=True) for _ in range(1000)])
two_way_deltas1.mean(), two_way_deltas1.min(), two_way_deltas1.max()

(np.float64(2.684), np.int64(-179), np.int64(180))

In [1219]:
pos_deltas1 = pd.Series([drop_time_hanler1.get_time_delta(two_way=False) for _ in range(1000)])
pos_deltas1.mean(), pos_deltas1.min(), pos_deltas1.max()

(np.float64(105.49), np.int64(30), np.int64(180))

In [1220]:
for _ in range(3):
    drop_time_hanler1.txns_count(receive=True, reset=False)
    drop_time_hanler1.txns_count(receive=False, reset=False)

In [1221]:
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns

(3, 3)

In [1222]:
drop_time_hanler1.txns_count(receive=True, reset=True)
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns

(1, 0)

In [1223]:
drop_time_hanler1.in_txns = 2
drop_time_hanler1.out_txns = 4
drop_time_hanler1.txns_count(receive=True, reset=True)
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns

(1, 0)

**`DropTimeHandler.get_txn_time` тест**  
Тест-кейсы
- Если это самая первая транзакция. Т.к. активность дропа начинается с входящей транзакции
- Не первая транзакция и достигнут лимит входящих транзакций для периода активности
- Не первая транзакция и достигнут лимит исходящих транзакций для периода активности
- Не первая транзакция. Лимиты не достигнуты. Транзакция входящая
- Не первая транзакция. Лимиты не достигнуты. Транзакция исходящая 

In [1224]:
start_time = pd.to_datetime("2025-07-02 11:15:00", format="%Y-%m-%d %H:%M:%S")
start_unix = pd_timestamp_to_unix(start_time)
start_time, start_unix

(Timestamp('2025-07-02 11:15:00'), 1751454900)

In [1225]:
last_time = pd.to_datetime("2025-07-02 18:31:00", format="%Y-%m-%d %H:%M:%S")
last_unix = pd_timestamp_to_unix(last_time)
last_time, last_unix

(Timestamp('2025-07-02 18:31:00'), 1751481060)

1. Если это самая первая транзакция. Т.к. активность дропа начинается с входящей транзакции

In [1226]:
drop_time_hanler1.reset_cache()
drop_time_hanler1.in_txns = 0
drop_time_hanler1.out_txns = 0
droptime1, dropunix1 = drop_time_hanler1.get_txn_time(receive=True, in_txns=0)
droptime1, dropunix1

(Timestamp('2025-01-11 01:06:00'), np.int64(1736557560))

In [1227]:
drop_time_hanler1.last_unix, drop_time_hanler1.in_txns, drop_time_hanler1.out_txns

(np.int64(1736557560), 1, 0)

2. Не первая транзакция и достигнут лимит входящих транзакций для периода активности
- текущая входящая - receive=True
- текущая исходящая - receive=False

In [1228]:
start_time, last_time

(Timestamp('2025-07-02 11:15:00'), Timestamp('2025-07-02 18:31:00'))

In [1229]:
drop_time_hanler1.reset_cache()
drop_time_hanler1.in_txns = drop_time_hanler1.in_lim
drop_time_hanler1.out_txns = drop_time_hanler1.out_lim - 1
drop_time_hanler1.start_unix = start_unix
drop_time_hanler1.last_unix = last_unix
droptime2, dropunix2 = drop_time_hanler1.get_txn_time(receive=True, in_txns=drop_time_hanler1.in_lim * 2)
droptime_start2 = pd.to_datetime(drop_time_hanler1.start_unix, unit="s")
droptime2, dropunix2, droptime_start2

(Timestamp('2025-07-03 11:28:00'),
 1751542080,
 Timestamp('2025-07-03 11:28:00'))

In [1230]:
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns, drop_time_hanler1.start_unix, drop_time_hanler1.last_unix

(1, 0, 1751542080, 1751542080)

In [1231]:
droptime2 - start_time

Timedelta('1 days 00:13:00')

3. Не первая транзакция и достигнут лимит исходящих транзакций для периода активности  
- текущая входящая - receive=True
- текущая исходящая - receive=False

In [1232]:
drop_time_hanler1.reset_cache()
drop_time_hanler1.in_txns = drop_time_hanler1.in_lim - 1
drop_time_hanler1.out_txns = drop_time_hanler1.out_lim
drop_time_hanler1.start_unix = start_unix
drop_time_hanler1.last_unix = last_unix
droptime3, dropunix3 = drop_time_hanler1.get_txn_time(receive=False, in_txns=drop_time_hanler1.in_lim * 2)
droptime_start3 = pd.to_datetime(drop_time_hanler1.start_unix, unit="s")
droptime3, dropunix3, droptime_start3

(Timestamp('2025-07-03 14:10:00'),
 1751551800,
 Timestamp('2025-07-03 14:10:00'))

In [1233]:
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns, drop_time_hanler1.start_unix, drop_time_hanler1.last_unix

(0, 1, 1751551800, 1751551800)

In [1234]:
droptime3 - start_time

Timedelta('1 days 02:55:00')

4. Не первая транзакция. Лимиты не достигнуты. Транзакция **входящая**

In [1235]:
drop_time_hanler1.reset_cache()
drop_time_hanler1.in_txns = drop_time_hanler1.in_lim - 1
drop_time_hanler1.out_txns = drop_time_hanler1.out_lim - 1
print(f"In txns: {drop_time_hanler1.in_txns} \nOut txns: {drop_time_hanler1.out_txns}")
drop_time_hanler1.start_unix = start_unix
drop_time_hanler1.last_unix = last_unix
droptime4, dropunix4 = drop_time_hanler1.get_txn_time(receive=True, in_txns=drop_time_hanler1.in_lim * 2)
droptime_start4 = pd.to_datetime(drop_time_hanler1.start_unix, unit="s")
droptime4, dropunix4, droptime_start4

In txns: 1 
Out txns: 4


(Timestamp('2025-07-02 20:02:00'),
 1751486520,
 Timestamp('2025-07-02 11:15:00'))

In [1236]:
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns, drop_time_hanler1.start_unix, drop_time_hanler1.last_unix

(2, 4, 1751454900, 1751486520)

In [1237]:
droptime4 - last_time

Timedelta('0 days 01:31:00')

5. Не первая транзакция. Лимиты не достигнуты. Транзакция **исходящая**

In [1238]:
drop_time_hanler1.reset_cache()
drop_time_hanler1.in_txns = drop_time_hanler1.in_lim - 1
drop_time_hanler1.out_txns = drop_time_hanler1.out_lim - 1
print(f"In txns: {drop_time_hanler1.in_txns} \nOut txns: {drop_time_hanler1.out_txns}")
drop_time_hanler1.start_unix = start_unix
drop_time_hanler1.last_unix = last_unix
droptime5, dropunix5 = drop_time_hanler1.get_txn_time(receive=False, in_txns=drop_time_hanler1.in_lim * 2)
droptime_start5 = pd.to_datetime(drop_time_hanler1.start_unix, unit="s")
droptime5, dropunix5, droptime_start5

In txns: 1 
Out txns: 4


(Timestamp('2025-07-02 21:10:00'),
 1751490600,
 Timestamp('2025-07-02 11:15:00'))

In [1239]:
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns, drop_time_hanler1.start_unix, drop_time_hanler1.last_unix

(1, 5, 1751454900, 1751490600)

In [1240]:
droptime5 - last_time

Timedelta('0 days 02:39:00')

## Класс `DistBehaviorHandler`  
Управление поведением дропов распределителей

In [ ]:
# ---------------------------- ВСТАВИТЬ КЛАСС ----------------------------------

**Тест DistBehaviorHandler**

In [1241]:
import data_generator.fraud.drops.behavior
import data_generator.indev
import data_generator.configs

importlib.reload(data_generator.indev)
importlib.reload(data_generator.configs)
importlib.reload(data_generator.fraud.drops.behavior)

from data_generator.indev import DropConfigBuilder
from data_generator.configs import DropDistributorCfg
from data_generator.fraud.drops.behavior import DistBehaviorHandler

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
dist_configs = drop_cfg_build.build_dist_cfg()
drop_amt1.reset_cache()
dist_behav1 = DistBehaviorHandler(configs=dist_configs, amt_hand=drop_amt1)

**`sample_scenario`**

`split_rate` = 0.7  
Тест `large_balance`: баланс больше чем `DistBehaviorHandler.trf_max`

In [1242]:
# Вероятности сценариев для large_balance
split_rate = 0.7
p_atm_and_trf = split_rate * 0.5
p_split_trf = split_rate * 0.5
p_atm = 1 - split_rate
print(f"""atm+transfer prob: {p_atm_and_trf}
split_transfer prob: {p_split_trf}
atm prob: {p_atm:.3f}""")
assert sum([p_atm_and_trf, p_split_trf, p_atm]) == 1

atm+transfer prob: 0.35
split_transfer prob: 0.35
atm prob: 0.300


In [1243]:
all_scens1 = []
i = 0
while i < 2000:
    drop_amt1.reset_cache()
    drop_amt1.balance = dist_behav1.trf_max + 1000
    assert drop_amt1.balance > dist_behav1.trf_max, "Balance is below trf_max"
    dist_behav1.sample_scenario()
    # print(dist_behav1.scen)
    all_scens1.append(dist_behav1.scen)
    i += 1
all_scens_ser1 = pd.Series(all_scens1)

In [1244]:
all_scens_ser1.value_counts(normalize=True)

atm+transfer      0.349
split_transfer    0.345
atm               0.306
Name: proportion, dtype: float64

`split_rate` = 0.7  
Тест `atm_eligible`: баланс больше чем `DistBehaviorHandler.atm_min`, но меньше чем `DistBehaviorHandler.trf_max`

In [1245]:
# Вероятности сценариев для atm_eligible
split_rate = 0.7
p_atm_and_trf = split_rate * 0.5
p_split_trf = split_rate * 0.5
p_trf = (1 - split_rate) * 0.5
p_atm = (1 - split_rate) * 0.5
print(f"""atm+transfer prob: {p_atm_and_trf}
split_transfer prob: {p_split_trf}
transfer prob: {round(p_trf, 3)}
atm prob: {round(p_atm, 3)}""")
assert sum([p_atm_and_trf, p_split_trf, p_trf, p_atm]) == 1

atm+transfer prob: 0.35
split_transfer prob: 0.35
transfer prob: 0.15
atm prob: 0.15


In [1246]:
all_scens2 = []
i = 0
while i < 2000:
    drop_amt1.reset_cache()
    drop_amt1.balance = dist_behav1.atm_min + 1000
    assert drop_amt1.balance < dist_behav1.trf_max, "Balance exceeds trf_max"
    dist_behav1.sample_scenario()
    all_scens2.append(dist_behav1.scen)
    i += 1
all_scens_ser2 = pd.Series(all_scens2)

In [1247]:
all_scens_ser2.value_counts(normalize=True)

atm+transfer      0.3465
split_transfer    0.3455
atm               0.1600
transfer          0.1480
Name: proportion, dtype: float64

`split_rate` = 0.7  
Тест `split_eligible`: баланс больше чем `DistBehaviorHandler.trf_min * 2`,  
но меньше чем `DistBehaviorHandler.atm_min` и `DistBehaviorHandler.trf_max`

In [1248]:
# Вероятности сценариев для split_eligible
split_rate = 0.7
p_split_trf = split_rate
p_trf = 1 - split_rate
print(f"""split_transfer prob: {p_split_trf}
transfer prob: {round(p_trf, 3)}""")
assert sum([p_split_trf, p_trf]) == 1

split_transfer prob: 0.7
transfer prob: 0.3


In [1249]:
all_scens3 = []
i = 0
while i < 3000:
    drop_amt1.reset_cache()
    drop_amt1.balance = dist_behav1.trf_min * 2
    assert drop_amt1.balance < dist_behav1.atm_min, "Balance exceeds atm_min"
    assert drop_amt1.balance < dist_behav1.trf_max, "Balance exceeds trf_max"
    dist_behav1.sample_scenario()
    all_scens3.append(dist_behav1.scen)
    i += 1
all_scens_ser3 = pd.Series(all_scens3)

In [1250]:
all_scens_ser3.value_counts(normalize=True)

split_transfer    0.707333
transfer          0.292667
Name: proportion, dtype: float64

`split_rate` = 0.7  
Остальные случаи. баланс меньше чем `DistBehaviorHandler.trf_min * 2`

In [1251]:
all_scens4 = []
i = 0
while i < 3000:
    drop_amt1.reset_cache()
    drop_amt1.balance = dist_behav1.trf_min + 1000
    assert drop_amt1.balance < dist_behav1.trf_min * 2, "Balance exceeds the limit"
    dist_behav1.sample_scenario()
    all_scens4.append(dist_behav1.scen)
    i += 1
all_scens_ser4 = pd.Series(all_scens4)

In [1252]:
all_scens_ser4.value_counts(normalize=True)

transfer    1.0
Name: proportion, dtype: float64

**`in_chunks_val`**

In [1253]:
# Список сценариев
# "transfer"
# "atm"
# "atm+transfer"
# "split_transfer"

dist_behav1.scen = "split_transfer"
dist_behav1.in_chunks_val()
dist_behav1.in_chunks

True

**`guide_scenario`**

In [1256]:
# Список сценариев
# "atm+transfer"
# "atm"
# "split_transfer"
# "transfer"

dist_behav1.scen = "transfer"
dist_behav1.guide_scenario()
dist_behav1.online

True

**`to_drop`**

In [1257]:
print(dist_behav1.to_drop_rate)
to_drop_list1 = []
for _ in range(100):
    to_drop_list1.append(dist_behav1.to_drop)
to_drop_ser1 = pd.Series(to_drop_list1)
to_drop_ser1.value_counts(normalize=True)

0.1


False    0.92
True     0.08
Name: proportion, dtype: float64

**`to_crypto`**

In [1258]:
dist_behav1.online = True
print(dist_behav1.crypto_rate)
to_drop_list2 = []

for _ in range(100):
    to_drop_list2.append(dist_behav1.to_drop)
to_drop_ser2 = pd.Series(to_drop_list2)
to_drop_ser2.value_counts(normalize=True)

0.1


False    0.91
True     0.09
Name: proportion, dtype: float64

**`stop_after_decline`**

In [424]:
dist_behav1.attempts = 2
dist_behav1.stop_after_decline(declined=True)

False

**`reset_cache`**

In [1261]:
dist_behav1.attempts = 3
dist_behav1.scen = "atm"
dist_behav1.online = True
dist_behav1.in_chunks = False
dist_behav1.reset_cache(all=True)
dist_behav1.attempts, dist_behav1.scen, dist_behav1.online, dist_behav1.in_chunks

(0, None, None, None)

**`attempts_after_decline`**  
self.online = True, self.online = False

In [1264]:
att_vals1 = []
dist_behav1.online = True

for _ in range(100):
    dist_behav1.attempts_after_decline(declined=True)
    att_vals1.append(dist_behav1.attempts)
att_vals_ser1 = pd.Series(att_vals1)

In [1265]:
att_vals_ser1.agg(["min","mean", "max"])

min     0.00
mean    1.94
max     4.00
dtype: float64

**`deduct_attempts`**

In [434]:
dist_behav1.attempts = 1
dist_behav1.deduct_attempts(declined=True, receive=True)
dist_behav1.attempts

1

**Расчет вероятностей сценариев в `DistBehaviorHandler`**

In [ ]:
# inbound_amt: 
#   low: 5000
#   high: 100000
#   mean: 30000
#   std: 20000

In [308]:
# Баланс генерируется из обрезанного нормального распределения
in_low1 = 5000
in_high1 = 100000
mean1 = 30000
std1 = 20000
a1 = (in_low1 - mean1) / std1
b1 = (in_high1 - mean1) / std1

split_rate = 0.7
# Есть лимиты для сумм операций
atm_min1 = 10000 # минимальная сумма для снятия. Частями или целиком определяется split_rate
trf_lim = 40000 # максимальная сумма для перевода. Если больше - дробим на части
trf_min = 3000*2 # минимальная сумма для перевода по частям. Если меньше - переводим целиком

In [309]:
# вероятность что сумма будет больше trf_lim
p_trf_lim = round(truncnorm(a=a1, b=b1, loc=mean1, scale=std1).sf(x=trf_lim), 3)
p_trf_lim

np.float64(0.345)

In [310]:
# Вероятность что сумма больше atm_min1 но меньше trf_lim
p_atm_min = round(truncnorm(a=a1, b=b1, loc=mean1, scale=std1).sf(x=atm_min1) - p_trf_lim, 3)
p_atm_min

np.float64(0.596)

In [311]:
# вероятность что сумма больше trf_min, но меньше trf_lim и atm_min1
p_trf_min = round(truncnorm(a=a1, b=b1, loc=mean1, scale=std1).sf(x=trf_min) - p_trf_lim - p_atm_min, 3)
p_trf_min

np.float64(0.048)

In [312]:
split_rate = 0.7 # доля когда будем переводить частями

(p_trf_lim + p_atm_min + p_trf_min) * split_rate

np.float64(0.6922999999999999)

In [313]:
round(truncnorm(a=a1, b=b1, loc=mean1, scale=std1).sf(x=trf_min), 3) * split_rate

np.float64(0.6922999999999999)

In [315]:

# conditions = [
#             (balance > self.trf_lim, ["atm","split_transfer", "atm+transfer"]),
#             (balance >= self.atm_min, ["transfer", "atm", "split_transfer", "atm+transfer"]),
#             (balance >= self.trf_min * 2, ["transfer","split_transfer"]),
#             "transfer",
#         ]
# Вероятность случаев перевода частями если по условиям выше (синтаксически неправильно) и без split_rate
p_trf_lim * 2/3 + p_atm_min * 1/2 + p_trf_min * 1/2

np.float64(0.552)

In [645]:

# conditions = [
#             (balance > self.trf_lim, ["atm","split_transfer", "atm+transfer"]),
#             (balance >= self.atm_min, ["transfer", "atm", "split_transfer", "atm+transfer"]),
#             (balance >= self.trf_min * 2, ["transfer","split_transfer"]),
#             "transfer",
#         ]
# Вероятность случаев перевода в крипту по условиям выше (синтаксически неправильно)
crypto_rate = 0.14
(p_trf_lim * 2/3 + p_atm_min * 3/4 + p_trf_min) * crypto_rate

np.float64(0.1015)

## Конец информативной части

# Старый код, функции и т.д.

## Класс `DropClient`
- метод get_account(self): Мб добавить вариант to_drop: bool. Перевод другому имеющемуся дропу. при этом. 
    - записываем счет дропа в свой счет. Атрибут account
    - записываем  счет дропа в атрибут drop_accounts
    - если надо перевести другому дропу, то фильтруем drop_accounts исключая свой
    - надо тогда клиентам приписать шестизначные счета. просто по порядку.
    - счета куда отправляют дропы это номера начинающиеся со счета последнего клиента + 1

In [ ]:
# class DropClient: 
#     """
#     Генератор сумм входящих/исходящих транзакций, сумм снятий, номеров счетов.
#     Управление балансом текущего дропа.
#     """
#     def __init__(self, accounts, account, outer_accounts, balance=0, batch_txns=0, chunk_size=0, used_accounts=pd.Series(name="account_id")):
#         """
#         accounts - pd.DataFrame. Счета клиентов банка.
#         account - int. Номер счета текущего дропа.
#         outer_accounts - pd.Series. Номера счетов для входящих и исходящих переводов в/из других банков.
#         balance - float. Текущий баланс дропа
#         batch_txns - int. Счетчик транзакций сделанных в рамках распределения полученной партии денег
#         chunk_size - int, float. Последний созданный размер части баланса для перевода по частям.
#         used_accounts - pd.Series. Счета на которые дропы уже отправляли деньги.
#         """
#         self.accounts = accounts
#         self.account = account
#         self.outer_accounts = outer_accounts
#         self.balance = balance
#         self.batch_txns = batch_txns
#         self.chunk_size = chunk_size
#         self.used_accounts = used_accounts

#     def update_balance(self, amount, add=False, declined=False):
#         """
#         Увеличить/уменьшить баланс на указанную сумму
#         -------------------
#         amount - float, int.
#         add - bool. Прибавлять сумму или отнимать.
#         declined - bool. Отклонена ли транзакция или одобрена.
#         """
#         # Не обновлять баланс если транзакция отклонена.
#         if declined:
#             return
            
#         # Увеличить баланс   
#         if add:
#             self.balance += amount
#             return
            
#         # Уменьшить баланс    
#         self.balance -= amount

#     def receive(self, declined, low=5000, high=100000, mean=30000, std=20000, round=500):
#         """
#         Генерация суммы входящего перевода
#         --------------------------
#         declined - bool. Отклонена ли транзакция или одобрена
#         low - float. Минимальная сумма
#         high - float. Максимальная сумма
#         mean - float. Средняя сумма
#         std - float. Стандартное отклонение
#         round - int. Округление целой части. По умолчанию 500. Значит числа будут либо с 500 либо с 000 на конце
#                      При условии что round не больше low и high. Чтобы отменить округление, нужно выставить 1
#         """

#         # Генерация суммы. Округление целой части при необходимости
#         amount = get_values_from_truncnorm(low_bound=low, high_bound=high, mean=mean, std=std)[0] // round * round
        
#         # Обновляем баланс если транзакция не отклонена
#         self.update_balance(amount=amount, add=True, declined=declined)
        
#         return amount

#     def get_chunk_size(self, online=False, atm_min=10000, atm_share=0.5, round=500, rand_rate=0.9, start=0, stop=0, step=0):
#         """
#         Вернуть случайный размер суммы перевода для перевода по частям
#         либо вернуть долю от баланса для снятия/перевода по частям.
#         -------------------------------
#         online - bool. Онлайн или оффлайн. Перевод или банкомат. Если банкомат, то снимается доля atm_share от баланса, но не меньше atm_min
#         atm_min - int, float. Минимальная сумма снятия дропом в банкомате.
#         atm_share - float. Доля от баланса если снятие через банкомат.
#         round - int. Округление целой части. По умолчанию 500. 
#                      Значит суммы будут округлены до тысяч или пяти сотен
#         rand_rate - float. От 0 до 1. Процент случаев, когда каждый НЕ первый чанк будет случайным и не зависеть от предыдущего.
#                            Но возможны случайные совпадаения с предыдущим размером чанка.
#                            Доля случайных размеров подряд будет:
#                            p(c) - вероятность взять определенный размер (зависит от размера выборки чанков)
#                            p(rr) - rand_rate
#                            p(rr) - (p(rr) * p(c)). Например p(rr) = 0.9, 5 вариантов размеров чанка - p(c) = 0.20
#                            0.9 - (0.9 * 0.2) = 0.72
#                            В около 72% случаев размеры чанков не будут подряд одинаковыми 
#         start - int. Минимальный размер. Прописываем если генерация не через share.
#                      То же самое для stop и step
#         stop - int. Максимальный размер - не входит в возможный выбор.
#                     Максимальное генерируемое значение равно stop - step
#         step - int. Шаг размеров.
#         --------------------
#         Возвращает np.int64
#         Результат кэшируется в self.chunk_size
#         """
#         # Если это не первая транзакция в серии транзакции для одной полученной дропом суммы
#         # И случайное число больше rand_rate, то просто возвращаем ранее созданный размер чанка
#         if self.batch_txns != 0 and np.random.uniform(0,1) > rand_rate:
#             return self.chunk_size

#         # Если перевод
#         if online:
#             sampling_array = np.arange(start, stop, step)
#             self.chunk_size = np.random.choice(sampling_array)
#             return self.chunk_size
            
#         # Если снятие    
#         self.chunk_size = max(atm_min, self.balance * atm_share // round * round)
#         return self.chunk_size
            
        
#     def one_operation(self, amount=0, declined=False, in_chunks=False):
#         """
#         Генерация суммы операции дропа.
#         ---------
#         amount - float, int. Сумма перевода если перевод по частям - in_chunks == True
#         declined - bool. Отклонена ли транзакция или одобрена
#         in_chunks - bool. Перевод по частям или целиком. Если False, то просто пробуем перевести все с баланса
#                           При True нужно указать amount.
#         """
#         if in_chunks and amount <= 0:
#             raise ValueError(f"""If in_chunks is True, then amount must be greater than 0.
# Passed amount: {amount}""")

#         # Если перевод не по частям. Пробуем перевестит все с баланса. 
#         if not in_chunks:
#             amount = self.balance
#             self.update_balance(amount=self.balance, add=False, declined=declined)
#             # Прибавляем счетчик транзакции для текущей партии денег
#             self.batch_txns += 1
#             return amount

#         # Иначе считаем сколько частей исходя из размера одной части
#         chunks = self.balance // amount

#         # Если целое число частей больше 0. Пробуем перевести одну часть
#         if chunks > 0:
#             self.update_balance(amount=amount, add=False, declined=declined)
#             self.batch_txns += 1
#             return amount

#         # Если баланс меньше одной части. Пробуем перевести то что осталось
#         rest = self.balance
#         self.update_balance(amount=rest, add=False, declined=declined)
#         self.batch_txns += 1
#         return rest

#     def get_account(self, to_drop):
#         """
#         Номер счета входящего/исходящего перевода
#         to_drop - bool. Перевод другому дропу в нашем банке или нет.
#         """
#         # Фильтруем accounts исключая свой счет и выбирая дропов. Для случая если to_drop
#         drop_accounts = self.accounts.loc[(self.accounts.account_id != self.account) & (self.accounts.is_drop == True)]

#         # Если надо отправить другому дропу в нашем банке. При условии что есть другие дропы на текущий момент
#         if to_drop and not drop_accounts.empty: 
#             account = drop_accounts.account_id.sample(1).iat[0]
#             # Добавляем этот счет в использованные как последнюю запись в серии
#             self.used_accounts.loc[self.used_accounts.shape[0]] = account
#             return account

#         # Если отправляем/получаем из другого банка.  
#         # Семплируем номер внешнего счета который еще не использовался
#         account = self.outer_accounts.loc[~(self.outer_accounts.isin(self.used_accounts))].sample(1).iat[0]
#         # Добавляем этот счет в использованные как последнюю запись в серии
#         self.used_accounts.loc[self.used_accounts.shape[0]] = account
        
#         return account

#     def reset_cache(self, balance=True, used_accounts=True, chunk_size=True, batch_txns=True):
#         """
#         Сброс кэшированных значений
#         По умолчанию сбрасывается всё. Если что-то надо оставить, то надо выставить False
#         для этого
#         -----------------
#         balance - bool
#         used_accounts - bool
#         chunk_size - bool
#         batch_txns - bool
#         """
#         if balance:
#             self.balance = 0
#         if used_accounts:
#             self.used_accounts = pd.Series(name="account_id")
#         if chunk_size:
#             self.chunk_size = 0
#         if batch_txns:
#             self.batch_txns = 0